## Loading Datasets

In [8]:
import pandas as pd 
import numpy as np
import re
import gender_guesser.detector as gender

In [9]:
real_users = pd.read_csv('datasets/realusers.csv')
fake_users = pd.read_csv('datasets/fakeusers.csv')

In [10]:
print(real_users.shape)
print(fake_users.shape)

(1481, 41)
(1337, 41)


## Data Preprocessing 
#### Concatenate both datasets

In [11]:
# Concatenate(Join) both datasets into a single dataset
X = pd.concat([real_users, fake_users])
X

,Unnamed: 0,id,name,screen_name,statuses_count,followers_count,friends_count,favourites_count,listed_count,created_at,...,verified,description,updated,dataset,age_in_days,ratio statuses_count/age,ratio Favorites/age,ratio Friends/Followers,length_of_bio,reputation
0,0,3610511,Davide Dellacasa,braddd,20370,5470,2385,145,52,2007-04-06 10:58:22+00:00,...,NaN,Founder of http://www.screenweek.it & http://w...,2015-02-14 10:54:49,E13,5822,3.499,0.025,0.436,151,0.696372
1,1,5656162,Simone Economo,eKoeS,3131,506,381,9,40,2007-04-30 15:08:42+00:00,...,NaN,BSc degree (cum laude) in Computer Engineering...,2015-02-14 10:54:49,E13,5797,0.540,0.002,0.753,104,0.570462
2,2,5682702,tacone,tacone_,4024,264,87,323,16,2007-05-01 11:53:40+00:00,...,NaN,Cogito ergo bestemmio.,2015-02-14 10:54:49,E13,5797,0.694,0.056,0.330,22,0.752137
3,3,6067292,alesaura,alesstar,40586,640,622,1118,32,2007-05-15 16:55:16+00:00,...,NaN,"Se la vita ti dà sarde, scapocciale!",2015-02-14 10:54:49,E13,5782,7.019,0.193,0.972,36,0.507132
4,4,6015122,Angelo,PerDiletto,2016,62,64,13,0,2007-05-13 19:52:00+00:00,...,NaN,Je me souviens,2015-02-14 10:54:49,E13,5784,0.349,0.002,1.032,14,0.492063
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1332,1332,1391497074,Verda Marks,VerdaMarks1,1,0,17,0,0,2013-04-30 08:23:57+00:00,...,NaN,I have been in business onlin and offline for ...,2015-02-14 10:40:01,INT,3606,0.000,0.000,inf,158,0.000000
1333,1333,1391544607,Danial Campbell,DanialCampbell2,0,1,17,0,0,2013-04-30 08:34:49+00:00,...,NaN,NaN,2015-02-14 10:40:01,INT,3606,0.000,0.000,17.000,3,0.055556
1334,1334,1391622127,Maudie Meyer,MaudieMeyer1,2,0,15,0,0,2013-04-30 09:21:12+00:00,...,NaN,NaN,2015-02-14 10:40:01,INT,3606,0.001,0.000,inf,3,0.000000
1335,1335,1391832212,Harriett Harvey,HarriettHarvey9,2,0,16,0,0,2013-04-30 11:25:11+00:00,...,NaN,NaN,2015-02-14 10:40:01,INT,3606,0.001,0.000,inf,3,0.000000


In [12]:
X.columns

Index(['Unnamed: 0', 'id', 'name', 'screen_name', 'statuses_count',
       'followers_count', 'friends_count', 'favourites_count', 'listed_count',
       'created_at', 'url', 'lang', 'time_zone', 'location', 'default_profile',
       'default_profile_image', 'geo_enabled', 'profile_image_url',
       'profile_banner_url', 'profile_use_background_image',
       'profile_background_image_url_https', 'profile_text_color',
       'profile_image_url_https', 'profile_sidebar_border_color',
       'profile_background_tile', 'profile_sidebar_fill_color',
       'profile_background_image_url', 'profile_background_color',
       'profile_link_color', 'utc_offset', 'protected', 'verified',
       'description', 'updated', 'dataset', 'age_in_days',
       'ratio statuses_count/age', 'ratio Favorites/age',
       'ratio Friends/Followers', 'length_of_bio', 'reputation'],
      dtype='object')

In [13]:
print('Missing sum values:')
columns_to_check = ['Unnamed: 0', 'id', 'name', 'screen_name', 'statuses_count',
       'followers_count', 'friends_count', 'favourites_count', 'listed_count',
       'created_at', 'url', 'lang', 'time_zone', 'location', 'default_profile',
       'default_profile_image', 'geo_enabled', 'profile_image_url',
       'profile_banner_url', 'profile_use_background_image',
       'profile_background_image_url_https', 'profile_text_color',
       'profile_image_url_https', 'profile_sidebar_border_color',
       'profile_background_tile', 'profile_sidebar_fill_color',
       'profile_background_image_url', 'profile_background_color',
       'profile_link_color', 'utc_offset', 'protected', 'verified',
       'description', 'updated', 'dataset', 'age_in_days',
       'ratio statuses_count/age', 'ratio Favorites/age',
       'ratio Friends/Followers', 'length_of_bio', 'reputation']

columns_to_remove = ['Unnamed: 0']

for column in columns_to_check:
    try:
        missing_count = X[column].isnull().sum()
        print(f"{column}: {missing_count}")
        
        # Check if missing count is greater than 2000
        if missing_count > 2000:
            columns_to_remove.append(column)
            print(f"Removing {column} due to more than 2000 missing values.")
    except KeyError:
        # Column having '/' might be causing the issue
        print(f"{column}: Column not found in the DataFrame.")

Missing sum values:
Unnamed: 0: 0
id: 0
name: 0
screen_name: 0
statuses_count: 0
followers_count: 0
friends_count: 0
favourites_count: 0
listed_count: 0
created_at: 0
url: 2355
Removing url due to more than 2000 missing values.
lang: 0
time_zone: 1749
location: 547
default_profile: 1090
default_profile_image: 2810
Removing default_profile_image due to more than 2000 missing values.
geo_enabled: 2097
Removing geo_enabled due to more than 2000 missing values.
profile_image_url: 0
profile_banner_url: 1831
profile_use_background_image: 58
profile_background_image_url_https: 0
profile_text_color: 0
profile_image_url_https: 0
profile_sidebar_border_color: 0
profile_background_tile: 2329
Removing profile_background_tile due to more than 2000 missing values.
profile_sidebar_fill_color: 0
profile_background_image_url: 0
profile_background_color: 0
profile_link_color: 0
utc_offset: 1749
protected: 2818
Removing protected due to more than 2000 missing values.
verified: 2818
Removing verified due 

In [14]:
# Remove columns with more than 2000 null values
X = X.drop(columns=columns_to_remove, axis=1)

# Display the DataFrame after removing columns
print("\nDataFrame after removing columns with more than 2000 null values:")
X


DataFrame after removing columns with more than 2000 null values:


,id,name,screen_name,statuses_count,followers_count,friends_count,favourites_count,listed_count,created_at,lang,...,utc_offset,description,updated,dataset,age_in_days,ratio statuses_count/age,ratio Favorites/age,ratio Friends/Followers,length_of_bio,reputation
0,3610511,Davide Dellacasa,braddd,20370,5470,2385,145,52,2007-04-06 10:58:22+00:00,it,...,3600.0,Founder of http://www.screenweek.it & http://w...,2015-02-14 10:54:49,E13,5822,3.499,0.025,0.436,151,0.696372
1,5656162,Simone Economo,eKoeS,3131,506,381,9,40,2007-04-30 15:08:42+00:00,en,...,3600.0,BSc degree (cum laude) in Computer Engineering...,2015-02-14 10:54:49,E13,5797,0.540,0.002,0.753,104,0.570462
2,5682702,tacone,tacone_,4024,264,87,323,16,2007-05-01 11:53:40+00:00,en,...,3600.0,Cogito ergo bestemmio.,2015-02-14 10:54:49,E13,5797,0.694,0.056,0.330,22,0.752137
3,6067292,alesaura,alesstar,40586,640,622,1118,32,2007-05-15 16:55:16+00:00,en,...,3600.0,"Se la vita ti dà sarde, scapocciale!",2015-02-14 10:54:49,E13,5782,7.019,0.193,0.972,36,0.507132
4,6015122,Angelo,PerDiletto,2016,62,64,13,0,2007-05-13 19:52:00+00:00,it,...,3600.0,Je me souviens,2015-02-14 10:54:49,E13,5784,0.349,0.002,1.032,14,0.492063
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1332,1391497074,Verda Marks,VerdaMarks1,1,0,17,0,0,2013-04-30 08:23:57+00:00,en,...,NaN,I have been in business onlin and offline for ...,2015-02-14 10:40:01,INT,3606,0.000,0.000,inf,158,0.000000
1333,1391544607,Danial Campbell,DanialCampbell2,0,1,17,0,0,2013-04-30 08:34:49+00:00,en,...,NaN,NaN,2015-02-14 10:40:01,INT,3606,0.000,0.000,17.000,3,0.055556
1334,1391622127,Maudie Meyer,MaudieMeyer1,2,0,15,0,0,2013-04-30 09:21:12+00:00,en,...,NaN,NaN,2015-02-14 10:40:01,INT,3606,0.001,0.000,inf,3,0.000000
1335,1391832212,Harriett Harvey,HarriettHarvey9,2,0,16,0,0,2013-04-30 11:25:11+00:00,en,...,NaN,NaN,2015-02-14 10:40:01,INT,3606,0.001,0.000,inf,3,0.000000


In [15]:
# Assigning False '0' to fake_users list and true '1' to real_users list
y = len(fake_users)*[0]+len(real_users)*[1]

#### Gender Detection Feature Extraction from First Name

In [16]:
# Create a detector instance
sex_predictor = gender.Detector(case_sensitive=False)

# Extract the first name and predict their genders
X['First Name'] = X['name'].str.split(' ').str.get(0)

# To handle names that have unrecognized characters
def clean_name(name):
    cleaned_name = re.sub(r'[^\x00-\x7F]+', '', name)
    return cleaned_name

# Clean the 'First Name' values
X['First Name'] = X['First Name'].apply(clean_name)

X['Predicted Sex'] = X['First Name'].apply(sex_predictor.get_gender)

# Mapping of Gender
sex_dict = {'female': -2, 'mostly_female': -1, 'unknown': 0, 'mostly_male': 1, 'male': 2}

# Handle 'unknown' values
X['Predicted Sex'] = X['Predicted Sex'].apply(lambda x: 'unknown' if x == 'andy' else x)

# Map the predicted genders to codes
X['Sex Code'] = X['Predicted Sex'].map(sex_dict).astype(int)

In [17]:
X.sample(5)

,id,name,screen_name,statuses_count,followers_count,friends_count,favourites_count,listed_count,created_at,lang,...,dataset,age_in_days,ratio statuses_count/age,ratio Favorites/age,ratio Friends/Followers,length_of_bio,reputation,First Name,Predicted Sex,Sex Code
288,616287324,Cassidy Becker,cassidyuhx,64,22,624,0,0,2012-06-23 15:32:50+00:00,en,...,INT,3916,0.016,0.00,28.364,156,0.034056,Cassidy,mostly_female,-1
59,616212355,Dusti Butler,dustibutlerzgv,66,20,722,0,0,2012-06-23 15:34:19+00:00,en,...,INT,3916,0.017,0.00,36.100,69,0.026954,Dusti,unknown,0
310,616298418,Vern Hester,vernnir,51,22,672,0,0,2012-06-23 15:45:43+00:00,en,...,INT,3916,0.013,0.00,30.545,151,0.031700,Vern,male,2
1204,1069178154,your curls,lwtomlinsonsass,4898,1498,1478,670,9,2013-01-07 21:02:56+00:00,it,...,E13,3718,1.317,0.18,0.987,46,0.503360,your,unknown,0
623,343473585,monica cianchini,moninka64,657,55,90,1,1,2011-07-27 16:44:42+00:00,it,...,E13,4248,0.155,0.00,1.636,31,0.379310,monica,female,-2


### Assigning numbering representation for each language

In [18]:
# Create a mapping of unique 'lang' values to codes
lang_list = list(enumerate(np.unique(X['lang'])))
lang_dict = {name: i for i, name in lang_list}

lang_dict

{'de': 0, 'en': 1, 'es': 2, 'fr': 3, 'gl': 4, 'it': 5, 'nl': 6, 'tr': 7}

In [19]:
# Map 'lang' values to 'lang_code' and convert to integers
X['lang_code'] = X['lang'].map(lambda lang: lang_dict[lang]).astype(int)

# Feature Extraction from columns
feature_columns_to_use = ['Sex Code','statuses_count','followers_count','friends_count','favourites_count','listed_count','lang_code']

# Create a new DataFrame with selected feature columns
X = X[feature_columns_to_use]

In [31]:
x = X 
x

,Sex Code,statuses_count,followers_count,friends_count,favourites_count,listed_count,lang_code
0,2,20370,5470,2385,145,52,5
1,-2,3131,506,381,9,40,1
2,0,4024,264,87,323,16,1
3,0,40586,640,622,1118,32,1
4,2,2016,62,64,13,0,5
...,...,...,...,...,...,...,...
1332,-2,1,0,17,0,0,1
1333,2,0,1,17,0,0,1
1334,-2,2,0,15,0,0,1
1335,-2,2,0,16,0,0,1
